Read .nc files

In [ ]:
import xarray as xr

# Open the NetCDF file
file_path = r"E:\IPMA\ERA5\Temperature\1raw_month_1979_2024\ERA5_hourly_temp_197901.nc"
ds = xr.open_dataset(file_path)

# Display the dataset's structure and metadata
print("Dataset Overview:\n")
print(ds)

# List all variables in the dataset
print("\nAvailable Variables:")
for var in ds.variables:
    print(f"- {var}: {ds[var].dims}, {ds[var].shape}")


Dataset Overview:

<xarray.Dataset> Size: 74MB
Dimensions:     (valid_time: 744, latitude: 129, longitude: 193)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 6kB 1979-01-01 ... 1979-01-31T23:...
  * latitude    (latitude) float64 1kB 66.0 65.75 65.5 65.25 ... 34.5 34.25 34.0
  * longitude   (longitude) float64 2kB -12.0 -11.75 -11.5 ... 35.5 35.75 36.0
    expver      (valid_time) <U4 12kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 74MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-16T11:35 GRIB to CDM+CF via cfgrib-0.9.1...

Available Variables:
- number: (), ()
- valid_time: ('valid_time',), (744,)
- latitude: ('latitude',), (129,)
- longitude: ('

Read .mat files

In [ ]:
from scipy.io import loadmat

data = loadmat(r"E:\IPMA\FRP\MODIS_2001.mat", struct_as_record=False, squeeze_me=True)

dados = data["dados"]  # Extract variable

print("Type of 'dados':", type(dados))

if hasattr(dados, "_fieldnames"):  # Check if it's a struct
    print("Struct fields:", dados._fieldnames)
    for field in dados._fieldnames:
        print(f"Field '{field}':", getattr(dados, field))

View on map

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load your NetCDF file (change the path accordingly)
file_path = r"E:\IPMA\SPEIbase_v2-10\spei01.nc"
ds = xr.open_dataset(file_path)

# Extract the data you want to plot (for example, pm10)
pm10_data = ds['spei'].isel(time=0)  # Select the first time step (change if needed)

# Extract longitude and latitude values
lon = ds['lon'].values
lat = ds['lat'].values

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Set up the map: add coastlines and borders
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the data (you can adjust the colormap and other settings)
c = ax.pcolormesh(lon, lat, pm10_data, cmap='viridis', shading='auto', transform=ccrs.PlateCarree())

# Show the plot
plt.show()

# Close the dataset after processing
ds.close()


In [4]:
import xarray as xr
import pandas as pd

# Open the NetCDF file
nc_file = r"E:\IPMA\ERA5\Temperature\2conversion_month_1979_2024\ERA5_hourly_temp_197901.nc"
ds = xr.open_dataset(nc_file)

# Convert all variables to a DataFrame
df = ds.to_dataframe().reset_index()

# Display the first few rows
print(df.head())


  valid_time  latitude  longitude  number expver       t2m
0 1979-01-01      66.0     -12.00       0   0001 -1.421967
1 1979-01-01      66.0     -11.75       0   0001 -1.427826
2 1979-01-01      66.0     -11.50       0   0001 -1.437592
3 1979-01-01      66.0     -11.25       0   0001 -1.439545
4 1979-01-01      66.0     -11.00       0   0001 -1.423920


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = r"E:\IPMA\RAQUEL\FRP\2020_2023.csv"

# Load CSV into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify data loading
print(df.head())

In [ ]:
import numpy as np

# Load the .npy file
file_path = r"E:\IPMA\RAQUEL\SPI\SPI3_1979.npy"
data = np.load(file_path)

# Print structure details
print("Shape:", data.shape)
print("Data type:", data.dtype)
print("Preview (first slice):\n", data[0])  # Print first "layer" of the 3D array
